## Page Content & Link & Description Dataframe (Beautifulsoup)

<br>

### Development Environment

In [2]:
import os
import re
import time
import requests
import urllib
import threading
import pandas as pd
from glob import glob
from collections import Counter
from bs4 import BeautifulSoup
from urllib.request import urlretrieve

### Page Information

In [33]:
def make_total_page_content_link_description_list(link_pattern, number_of_page):

    total_title_list = []
    total_link_list = []
    total_description_list = []

    # query = urllib.parse.quote('keyword')
    for page_num in range(number_of_page):
        url = link_pattern[0] + str(page_num) + link_pattern[1] # + query

        request = urllib.request.Request(link_pattern[0])


        response = urllib.request.urlopen(request)
        response_code = response.getcode()

        if response_code==200:
            response_body = response.read()
        else:
            print("Error Code:"+response_code)

        soup = BeautifulSoup(response_body, 'html.parser') 

        title_list, link_list, description_list = make_page_content_link_description_list(soup)

        total_title_list.append(title_list)
        total_link_list.append(link_list)
        total_description_list.append(description_list)

    total_title_list = sum(total_title_list, [])
    total_link_list = sum(total_link_list, [])
    total_description_list = sum(total_description_list, [])

    return total_title_list, total_link_list, total_description_list

### Make Page Content Link Description Dataframe

In [44]:
def make_page_content_link_description_list(soup):

    html_tags = ['div', 'img', 'description', 'thumbnail']
    title_list = []
    link_list = []
    description_list = []

    a = soup.select("a")
    for a_text in a:
        a_text = str(a_text)

        if 'detail' in a_text:
            a_title = re.search('/">(.+?)</a>', a_text)
            if a_title != None:
                a_title = a_title.group()
                if all(html_tag not in a_title for html_tag in html_tags):
                    a_title = a_title.replace('/">', '')
                    a_title = a_title.replace('</a>', '')
                    title_list.append(a_title)

            a_link = re.search('<a href=(.+?)/">', a_text)
            if a_link != None:
                a_link = a_link.group()
                a_link = a_link.replace('<a href="', 'https://yozm.wishket.com/')
                a_link = a_link.replace('">', '')
                link_list.append(a_link)
                link_list = dict.fromkeys(link_list)
                link_list = list(link_list)

        a_description = re.search('<p class="item-description">(.+?)</p>', a_text)
        if a_description != None:
            a_description = a_description.group()
            a_description = a_description.replace('<p class="item-description">', '')
            a_description = a_description.replace('</p>', '')
            description_list.append(a_description)

    return title_list, link_list, description_list

In [45]:
def make_page_content_link_description_df(titles, links, descriptions):
    
    page_content_link_description_df = pd.DataFrame({'title':titles, 'link':links, 'description':descriptions,})
    
    return page_content_link_description_df

In [46]:
link_pattern = ['https://yozm.wishket.com/magazine/list/develop/?page=', '&sort=&q=']
number_of_page = 59
title_list, link_list, description_list = make_total_page_content_link_description_list(link_pattern, number_of_page)

In [47]:
page_content_link_description_df = make_page_content_link_description_df(title_list, link_list, description_list)

In [48]:
pd.set_option('display.max_colwidth', None)
page_content_link_description_df 

,title,link,description
0,개발자가 알아야 할 데이터 지향 프로그래밍 with JDK 20,https://yozm.wishket.com//magazine/detail/2164/,"객체지향 프로그래밍(Object-Oriented Programming, OOP)은 클래스(Class)를 통해 데이터와 행위를 묶어 관리하는 프로그래밍 패러다임입니다. 그렇지만 OOP의 사용에 대한 모호함에 대한 논의는 끊임없이 진행되고 있습니다. 이러한 배경 속에서 예호나단 샤르빗(Yehonathan Sharvit)이 데이터 지향 프로그래밍(Data Oriented Programming)이라는 새로운 개념을 제안했습니다. 전편 ‘개발자가 알아야 할 데이터 지향 설계란?’에 이어, 이번 글에서는 데이터 지향 프로그래밍은 어떤 내용인지 알아보겠습니다."
1,좋은 객체 ID(Object ID) 만들기,https://yozm.wishket.com//magazine/detail/2161/,"객체 ID는 객체 지향 프로그래밍, 데이터베이스 관리, 네트워킹 등 다양한 컴퓨팅 환경에서 사용되는데요. 이 포스트에서는 좋은 객체 ID를 만들어야 하는 이유와 방법을 고유성과 식별 가능성을 중심으로 알아보겠습니다."
2,"스프링 데이터 JPA, 5분 만에 알아보기",https://yozm.wishket.com//magazine/detail/2160/,"스프링 데이터 JPA를 알려면 먼저 SQL을 몰라도 데이터베이스를 조작할 수 있게 해주는 편리한 도구인 ORM 개념을 알아야 합니다. 그러고 나서 JPA를 알아야 비로소 스프링 데이터 JPA를 알 수 있습니다. ORM은 SQL 공부 를 한 적이 있다면 그 편리함을 더 크게 느낄수 있는 도구인데요, 스프링 부트의 ORM 기술 표준인 스프링 JPA와 이를 구현하기 위한 하이버네이트를 사용해서 차례대로 알아보겠습니다."
3,개발자가 알아야 할 데이터 지향 설계란?,https://yozm.wishket.com//magazine/detail/2157/,"객체지향 프로그래밍은 클래스(Class)를 통해 데이터와 행위를 묶어 관리하는 프로그래밍 패러다임입니다. 그렇지만 OOP의 사용에 대한 모호함에 대한 논의는 여전히 진행되고 있으며, 최근에는 함수형 프로그래밍(Functional Programming, FP)을 통한 활용도 많은 관심을 받고 있습니다. 이러한 배경 속에서 예호나단 샤르빗(Yehonathan Sharvit)이 데이터 지향 프로그래밍(Data Oriented Programming)이라는 새로운 개념을 제안했습니다. 그래서 이번 글에서는 데이터 지향 설계와 데이터 지향 프로그래밍에 대해 간략히 알아보는 시간을 갖고자 합니다. 첫 번째로 데이터 지향 설계와 관련된 이야기입니다."
4,자바 생성 패턴을 실제 프로젝트에 적용하려면?,https://yozm.wishket.com//magazine/detail/2155/,"오늘은 자바 디자인 패턴 시리즈 세 번째 글로 이번 편에서는 생성 패턴(Creational Patterns)의 종류 및 프로젝트 적용 방법에 대해 알아보고자 합니다. 전편에서 살펴본 구조 패턴처럼 생성 패턴 역시 소프트웨어 설계에 있어 다양하게 활용되고 있습니다. 자바 생성 패턴 중 싱글턴(Singleton), 팩토리(Factory), 빌더(Builder), 프로토타입(Prototype) 패턴을 살펴보고, 실제 프로젝트에서 어떻게 적용하는지 참고해 보시길 바랍니다."
...,...,...,...
585,프론트엔드 개발자가 구글 I/O 2023 웹 세션에서 배운 것들,https://yozm.wishket.com//magazine/detail/2152/,"Google I/O는 구글에서 매년 5월 경 주관하는 개발자 컨퍼런스로, 구글에서 새롭게 발표하는 기술, 안드로이드 등의 최신 업데이트 사항들을 공유한다. 특히 올해는 구글이 ‘바드(Bard)’라는 인공지능 챗봇을 공개하면서 한국어를 우선 지원하겠다고 밝혀 큰 화제가 되기도 했다. 이외에도 모바일, 웹, AI, 클라우드 등의 다양한 분야와 이러한 기술들을 직접 학습해 보는 세션이 준비되었다. 이번 글에서는 그중 웹에 대해 다룬 주요 세션을 웹 프론트엔드 개발자 관점에서 살펴보고자 한다."
586,"‘프론트엔드냐, 백엔드냐’를 고민하는 당신에게",https://yozm.wishket.com//magazine/detail/2151/,프론트엔드를 해야 할지 백엔드를 해야 할지 모르겠어요. 개발을 막 시작하거나 몇 년 지나지 않은 주니어 개발자들과 대화를 하다 보면 심심치 않게 듣는 말이다. 그렇다고 꼭 경험이 적은 개발자만의 고민은 아니다. 경력이 많은 개발자들도 종종 이런 고민을 토로한다. 프랑스 철학자 장 폴 사르트르는 “인생은 BCD이다”라고 말했다. 태어나서(Birth) 죽는(Death) 순간까지 끊임없이 선택(Choice)하며 산다는 뜻이다. 삶은 선택의 연속이다. 그렇다면 어떻게 해야 더 나은 선택을 할 수 있을까?
587,베푸는 방법과 비판을 받아들이는 방법,https://yozm.wishket.com//magazine/detail/2150/,"우리는 적게는 3-4명, 많게는 10명 이상의 인원이 협업하면서 프로젝트를 진행합니다. 어떻게 하면 팀원들 간에 더 나은 협업을 할 수 있는지, 좋은 팀을 만들 수 있는지 고민이 한 번쯤은 있으셨으리라 생각합니다. 오늘은 협업의 기술이란 책에서 소개하는 'HRT와 베푸는 방법과 비판을 받아들이는 방법 모두 배우기'를 통해 좋은 협업, 좋은 팀을 만들기 위해 무엇이 필요한지에 대한 내용을 공유하고자 합니다."
588,쿠팡이 클라우드 기반 ‘데이터베이스 백업 플랫폼’을 만든 이유,https://yozm.wishket.com//magazine/detail/2146/,"쿠팡에서 데이터는 대체 불가능한 자원입니다. DBA 엔지니어들은 데이터베이스에 문제가 발생하는 것을 원천 차단하고자 많은 노력을 기울이지만, 예측 불가한 일로 인해 데이터에 문제가 생길 수 있습니다. 예를 들어, 사소한 물리 디스크 장애로 데이터세트가 오염될 수도 있고, 개발자 실수로 데이터가 영구적으로 삭제될 수도 있습니다. 이러한 잠재적 데이터 재해를 빠르고 효율적으로 해결하기 위하여, 쿠팡 데이터베이스 엔지니어링 팀은 전체 백업 프로세스를 간소화, 자동화하는 강력한 백업 및 복구 플랫폼을 개발하였습니다."


In [ ]:
page_content_link_description_df.to_excel('page_content_link_description_df.xlsx')

<br>

### Reference

<b>Github<b>
<br>cheris8
<br>[동적 웹 페이지 크롤링 with Python](https://cheris8.github.io/data%20analysis/DC-Dynamic-Webpage-Crawling/)

<br><b>Magazine<b>
<br>[요즘IT](https://yozm.wishket.com/magazine/)